## Load libraries

In [0]:
import os
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


## Working directories

In [0]:
print(os.getcwd())

/content


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


**directories and paths**

In [0]:
# Set directories

dirRawData = "/content/drive/My Drive/AML/Projects/AML_2019/RawData/"
dirPData = "/content/drive/My Drive/AML/Projects/AML_2019/PData/"
dirPOutput = "/content/drive/My Drive/AML/Projects/AML_2019/POutput/"

## Load data


In [0]:
df_all = pd.read_csv(dirRawData + 'train.csv', 
                     na_values = 'NA')
#df_all.head()

In [0]:
vars_all = df_all.columns.values
var_dep = ['target']

vars_notToUse = ['id']
vars_ind = [var for var in vars_all if var not in (var_dep + vars_notToUse )]



In [0]:
vars_ind_numeric = list(df_all[vars_ind].columns[df_all[vars_ind].dtypes != 'object'])
#vars_ind_numeric

In [0]:
vars_ind_categorical = df_all.columns[df_all.dtypes == 'object'].tolist()
#vars_ind_categorical 

## One hot using sklearn

In [0]:
vars_ind_onehot = []
df_all_onehot = df_all.copy()
for col in vars_ind_categorical:
    #print(col)
    enc_ = OneHotEncoder()
    enc_.fit(df_all[[col]])
    oh = enc_.transform(df_all[[col]]).toarray()
    idx_mostFreq = np.argmax(oh.sum(axis = 0)) 
    oh = np.delete(oh, idx_mostFreq, axis=1)
    oh_names = enc_.categories_[0]
    oh_names = np.delete(oh_names, idx_mostFreq)
    oh_names = col + '_X_' + oh_names
    df_oh = pd.DataFrame(data = oh, columns = oh_names)
    df_all_onehot = pd.concat([df_all_onehot, df_oh], axis = 1, sort = False,join = 'inner')
    del df_all_onehot[col]
    vars_ind_onehot.extend(oh_names)


In [13]:
df_all_onehot.shape

(11962, 485)

## Add fold variable

TODO

- Create an instance of a random number generator (np.random.RandomState) with a seed of 2018 and call it rng.  
- Use it to generate random integers (rng.randint()) between 0 and 10.  Generate as many numbers as there are rows of df_all 
- save this data to a new column in df_all called 'fold'

In [0]:
rng = np.random.RandomState(2018)

In [0]:
fold = rng.randint(0, 10, df_all_onehot.shape[0])

In [0]:

df_all_onehot['fold'] = fold

In [17]:
df_all_onehot.shape

(11962, 486)

## Dividing data into train, validation and test set

### Train set

In [0]:
idx_train = df_all_onehot.where(df_all_onehot['fold'].isin(np.arange(0,6)))

In [0]:
idx_train = idx_train.dropna()

In [22]:
idx_train.shape

(7201, 486)

### Validation set

In [0]:
idx_val = df_all_onehot.where(df_all_onehot['fold'].isin(np.arange(6,7)))

In [0]:
idx_val = idx_val.dropna()

In [25]:
idx_val.shape

(1213, 486)

### Test set

In [0]:
idx_test   = df_all_onehot.where(df_all_onehot['fold'].isin([8,9]))

In [0]:
idx_test = idx_test.dropna()

In [28]:
idx_test .shape

(2353, 486)

## Find the appropriate value of Alpha/lambda

## Fit logistic regression on the train dataset

### Data specs

In [0]:
vars_toUse = [var for var in (vars_ind_numeric + vars_ind_onehot)]

In [34]:
len(vars_toUse)

483

In [35]:
# 
print('numeric variables', len(vars_ind_numeric))
print('one hot', len(vars_ind_onehot))

numeric variables 107
one hot 376


In [40]:
X = idx_train[vars_toUse] # model matrix 
X.shape

(7201, 483)

In [0]:
y = idx_train[var_dep] # and target variable
y.shape

In [41]:
x_val = idx_val[vars_toUse]
x_val.shape

(1213, 483)

In [46]:
y_val = idx_val[var_dep]
y_val.shape

(1213, 1)

In [43]:
x_test = idx_test[vars_toUse]
x_test.shape

(2353, 483)

In [44]:
y_test = idx_test[var_dep] # and target variable
y_test.shape

(2353, 1)


        We need to scale the coefficients so that they all run over a similar scale of numbers . this will make comparison of coefficients easier.
        In this case, we can use the standardscaler function in sklearn to standarduse the values vefore fitting the values
        

### Standard Scaler

In [0]:


standardScaler_ = StandardScaler()
reg = standardScaler_.fit(X,y)
X = standardScaler_.transform(X) #Mean and standard deviation are then stored to be used on later data using the transform method.

In [51]:
reg.n_samples_seen_

7201

In [0]:
# scaled features: means 
#print("scaled features: means: ", np.round(X.mean(axis=0),5) )

# scaled features: variance
#print("scaled features: standard deviation: ", np.round(X.std(axis=0),5) )

### Fitting model

In [53]:
# Next, use pip to install this version of the H2O Python module.
!pip install http://h2o-release.s3.amazonaws.com/h2o/rel-yates/4/Python/h2o-3.24.0.4-py2.py3-none-any.whl


     |████████████████████████████████| 121.9MB 1.2MB/s 


In [0]:

import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

In [55]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.3" 2019-04-16; OpenJDK Runtime Environment (build 11.0.3+7-Ubuntu-1ubuntu218.04.1); OpenJDK 64-Bit Server VM (build 11.0.3+7-Ubuntu-1ubuntu218.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.6/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp75hjydqb
  JVM stdout: /tmp/tmp75hjydqb/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp75hjydqb/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,Etc/UTC
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.4
H2O cluster version age:,22 days
H2O cluster name:,H2O_from_python_unknownUser_bjs27f
H2O cluster total nodes:,1
H2O cluster free memory:,3 Gb
H2O cluster total cores:,2
H2O cluster allowed cores:,2
H2O cluster status:,"accepting new members, healthy"


In [69]:
 h2o_train = h2o.H2OFrame(idx_train,destination_frame = 'idx_train')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [70]:
 h2o_val = h2o.H2OFrame(idx_val,destination_frame = 'idx_val')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [71]:
 h2o_test = h2o.H2OFrame(idx_test,destination_frame = 'idx_test')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [0]:
glm_1 = H2OGeneralizedLinearEstimator(alpha = 1, family = 'binomial', link = 'logit', lambda_search = True)

In [73]:
glm_1.train(x =vars_toUse,
           y = 'target',
           training_frame = h2o_train)

glm Model Build progress: |███████████████████████████████████████████████| 100%


## Inspect the coefficients

In [74]:
df_coef = pd.Series(glm_1.coef())
df_coef = df_coef[np.abs(df_coef) > 0]
df_coef.head()

Intercept   -5.016436
v_num_1      0.035735
v_num_2     -0.004763
v_num_5     -0.039746
v_num_6      0.124928
dtype: float64

In [75]:
# some have been penalised to non existence
df_coef.shape

(292,)

In [0]:
#h2o.predict(object=model, newdata=prostate.test)

In [0]:
#predictions = h2o.predict(glm_1, newdata = h2o_train)

In [0]:
glm_1.std_coef_plot()

## Predictions on train, Validation and test set

## Calculate the error in the train, validation and test datasets

We should use the loglik loss function

### Loss function